# Collaborative Filtering Demonstration

In [1]:
import scipy.io
import scipy.optimize
import numpy as np

np.random.seed(7404)

## 1. Load Data from Files

In [2]:
file = open("movie_ids.txt")

movieList_ = []
for line in file.readlines():
    movieName = line.split()[1:]
    movieList_.append(' '.join(movieName))

# no. of movies in dataset
print(len(movieList_))
print(movieList_[:5])

1682
['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', 'Get Shorty (1995)', 'Copycat (1995)']


In [3]:
data = scipy.io.loadmat('ex8_movies.mat')
Y_ = data['Y'].T
R_ = data['R'].T

# demensions and sample contents
print(Y_.shape)
print(Y_[:5])
print(R_.shape)
print(R_[:5])

(943, 1682)
[[5 3 4 ..., 0 0 0]
 [4 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [4 3 0 ..., 0 0 0]]
(943, 1682)
[[1 1 1 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]]


## 2. Data Pre-processing

In [4]:
# shortlist top 100 most rated movies
top100 = np.sum(R_, axis=0).argsort()[-100:][::-1]
movieList = [movieList_[i] for i in top100]
print(len(movieList))
Y = Y_[:, top100]
R = R_[:, top100]
print(Y.shape)

# top 10 movies in our shortlist
movieList[:10]

100
(943, 100)


['Star Wars (1977)',
 'Contact (1997)',
 'Fargo (1996)',
 'Return of the Jedi (1983)',
 'Liar Liar (1997)',
 'English Patient, The (1996)',
 'Scream (1996)',
 'Toy Story (1995)',
 'Air Force One (1997)',
 'Independence Day (ID4) (1996)']

In [5]:
def normalize(Y, R):
    Ymean = np.sum(Y*R, axis=0) / np.sum(R, axis=0) # (n, 1)
    # auto broadcast (m, n) - (1, n)
    Ynorm = (Y - Ymean.reshape(1, -1)) * R
    return Ynorm

# normalize ratings to center on 0
Ynorm = normalize(Y, R)
print(Ynorm.shape)
print(Ynorm[0])

(943, 100)
[ 0.64150943  1.19646365  0.84448819  0.99211045 -0.         -0.         -0.
  1.12168142 -0.          0.56177156  0.74761905  0.71670702 -0.06091371
  0.20153061 -0.28974359 -1.7109375  -0.69312169  0.79564033  0.33972603
 -0.          1.16571429 -0.         -0.04464286  0.0694864   0.36809816
  0.82716049 -0.85358255  0.9335443  -0.          1.07301587  0.25412541
  1.06644518 -0.          1.10367893 -0.         -0.         -0.
 -0.15151515  0.99322034 -0.          0.55631399  1.221843   -0.21501706
  0.96563574  1.08965517  1.0528169   0.55477032  0.225       1.23571429
  0.06884058  0.83695652  0.86181818  1.51470588 -0.          0.61423221
 -0.         -0.          1.27969349 -0.          0.03088803 -0.79296875
  0.125       1.20784314 -0.         -0.66135458 -0.          0.16334661
  1.08366534 -0.         -0.7854251  -0.07723577 -0.55737705  1.18442623
  0.12757202 -0.         -0.05809129 -1.10833333 -0.          0.89539749
 -0.10041841 -1.84745763 -0.         -0.    

## 3. Parameters Initiation

In [6]:
# define demensions
# m - no. of users
# n - no. of movies
# k - no. of latent factors (hidden features)
m, n = Y.shape
k = 3

# initiate parameters with random numbers
# Theta - the users preference to hidden features
# X - the movies tendency to hidden features
Theta_init = np.random.randn(m, k)
X_init = np.random.randn(n, k)

# define L2 regularization constant
L2 = 2.0

# unroll parameters
params_init = np.append(Theta_init.reshape(-1), X_init.reshape(-1)) # (m x k) + (n x k)
params_init.shape

(3129,)

## 4. Define the Cost Function

In [7]:
def costFunction(params, Y, R, m, n, k, L2):
    
    # roll back parameters
    Theta = params[:(m*k)].reshape(m, k)
    X = params[(m*k):].reshape(n, k)

    # calculate cost (J)
    E = (Theta.dot(X.T) - Y) * R        # (m x n)
    # J = SSE + Regularization Terms
    J = 0.5 * np.sum(E**2) + 0.5 * L2 * (np.sum(Theta**2) + np.sum(X**2)) # scalar

    # calculate gradients
    dJ_dTheta = E.dot(X) + L2 * Theta   # (m x n) x (n x k) = (m x k)
    dJ_dX = E.T.dot(Theta) + L2 * X     # (n x m) x (m x k) = (n x k)

    # unroll gradients
    grads = np.append(dJ_dTheta.reshape(-1), dJ_dX.reshape(-1)) # (m + n) x k
    return J, grads


## 5. Minimize the Cost Function

In [8]:
result = scipy.optimize.minimize(fun = costFunction, 
                                 x0 = params_init, 
                                 args = (Ynorm, R, m, n, k, L2),                                 
                                 method='L-BFGS-B',
                                 jac=True,  # return gradients in cost function
                                 options={'disp': True, 'maxiter': 1000}
                                )
result

      fun: 9987.2951619243904
 hess_inv: <3129x3129 LbfgsInvHessProduct with dtype=float64>
      jac: array([  9.67622661e-05,  -1.64833720e-04,  -4.75883352e-04, ...,
        -6.43348488e-04,  -5.60441463e-04,  -9.89050378e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 86
      nit: 79
   status: 0
  success: True
        x: array([ 0.10519864, -0.31942417,  0.22365613, ..., -1.07501069,
       -0.58499031, -1.43149514])

In [9]:
params = result.x
# roll back the parameters
Theta = params[:(m*k)].reshape(m, k)
X = params[(m*k):].reshape(n, k)

## 6. Latent Factor Analysis

In [10]:
# list top 10 tendency movies in latent factor 0, 1 & 2
print([(movieList[i], '%.2f'%X[i,0]) for i in X[:,0].argsort()[-10:][::-1]])
print()
print([(movieList[i], '%.2f'%X[i,1]) for i in X[:,1].argsort()[-10:][::-1]])
print()
print([(movieList[i], '%.2f'%X[i,2]) for i in X[:,2].argsort()[-10:][::-1]])
print()

[('Clockwork Orange, A (1971)', '2.60'), ('Pulp Fiction (1994)', '2.44'), ('Leaving Las Vegas (1995)', '2.40'), ('Trainspotting (1996)', '2.18'), ('Apocalypse Now (1979)', '2.06'), ('Dead Man Walking (1995)', '1.89'), ('Godfather, The (1972)', '1.87'), ('Chasing Amy (1997)', '1.86'), ('Raising Arizona (1987)', '1.86'), ('GoodFellas (1990)', '1.78')]

[('English Patient, The (1996)', '0.87'), ('Evita (1996)', '0.44'), ('Dead Man Walking (1995)', '0.32'), ('Graduate, The (1967)', '0.29'), ('Leaving Las Vegas (1995)', '0.18'), ('To Kill a Mockingbird (1962)', '0.10'), ('Fargo (1996)', '0.09'), ('Full Monty, The (1997)', '0.04'), ('Sense and Sensibility (1995)', '0.02'), ('Birdcage, The (1996)', '-0.05')]

[('Star Wars (1977)', '0.95'), ('Alien (1979)', '0.90'), ('Return of the Jedi (1983)', '0.79'), ('2001: A Space Odyssey (1968)', '0.76'), ('Empire Strikes Back, The (1980)', '0.73'), ('Clockwork Orange, A (1971)', '0.67'), ('Monty Python and the Holy Grail (1974)', '0.67'), ('Blade Runne

## 7. Recommending Movies

In [11]:
# find the Terminator fan with least rating activities
# 31 - Terminator, The (1984)
# 38 - Terminator 2: Judgment Day (1991)
terminator_fans = np.where((Y[:, 31] >= 4) & (Y[:, 38] >= 4))[0]
uid = terminator_fans[np.sum(R[terminator_fans], axis=1).argsort()[0]]
uid

201

In [12]:
# list what he has rated
[(movieList[i], Y[uid,i]) for i in Y[uid].argsort()[-10:][::-1]]

[('Terminator, The (1984)', 4),
 ('Contact (1997)', 4),
 ('Terminator 2: Judgment Day (1991)', 4),
 ('Full Monty, The (1997)', 4),
 ('Toy Story (1995)', 3),
 ('Back to the Future (1985)', 3),
 ('E.T. the Extra-Terrestrial (1982)', 3),
 ('Empire Strikes Back, The (1980)', 3),
 ('Princess Bride, The (1987)', 2),
 ('Amadeus (1984)', 2)]

In [13]:
# what we suggest to him
[movieList[i] for i in Theta.dot(X.T)[uid].argsort()[-5:][::-1]]

['Conspiracy Theory (1997)',
 'Murder at 1600 (1997)',
 'Volcano (1997)',
 'Independence Day (ID4) (1996)',
 'Saint, The (1997)']